In [74]:
import pandas as pd
from app import db
from app import User, Dataset, Document, Label, Project, projectlabel_association, Annotation
from medcat.cdb import CDB
from medcat.utils.vocab import Vocab
from medcat.cat import CAT

path = '/Users/kawsarnoor/Desktop/cogstack/models/medmen/'
vocab = Vocab()
vocab.load_dict(path + 'vocab.dat')

cdb = CDB()
cdb.load_dict(path + 'cdb.dat')

In [ ]:
from elasticsearch import Elasticsearch 
es=Elasticsearch([{'host':'localhost','port':9200}])

In [ ]:
cat = CAT(cdb=cdb, vocab=vocab)

In [ ]:
dataset = Dataset.query.filter_by(id=3).all()[0]

In [ ]:
text = Document.query.filter_by(dataset=dataset).all()[0].text

In [ ]:
doc = cat.get_entities(text)

In [ ]:
df = pd.DataFrame(doc)
df

In [ ]:
for cui in list(set(list(df['cui']))):
    new_label = Label(label=cui, labelType='sentence')
    db.session.add(new_label)
    db.session.commit()

In [ ]:
for idx, row in df.iterrows():
    label = Label.query.filter_by(label=row['cui']).all()[0]
    new_annotation = Annotation(user_id=2, label=label, project_id=3, document_id=25, start_idx=row['start'], end_idx=row['end'], completed=False)
    db.session.add(new_annotation)
    db.session.commit()

In [ ]:
delanns = Annotation.query.filter_by(project_id=3).all()
for delann in delanns:
    db.session.delete(delann)
    db.session.commit()

In [73]:
for idx, row in df.iterrows():
    
    entry={"cui": row['cui'], 
           "tui": row['tui'],
          "pretty_name": row['pretty_name'],
          "type": row['type']}


    es.index(index='snomed_uk', doc_type='snomedterm', id=idx, body=entry)


In [80]:
entry_string = 'C0678257'

query = [{"index":'snomed_uk'},
        {"query":{"bool":
                    {"must":[{"query_string":
                                {"query": entry_string + '*',
                                "analyze_wildcard":True,
                                "default_field":"*"}}],
                    "filter":[],
                    "should":[],
                    "must_not":[]}}}]

results = es.msearch(query)['responses'][0]
results['hits']['hits'][0]['_source']

{'cui': 'C0678257',
 'tui': 'T170',
 'pretty_name': 'described',
 'type': 'Intellectual Product'}